In [1]:
url='https://www.freelancer.com'
access_token="M6X6wTeZ4JfyUgGJA5VrDDX5VflEpG"

In [2]:
import requests
import re
from bs4 import BeautifulSoup
import json
import pandas as pd
import numpy as np
from selenium import webdriver
import time
import os
import urllib.parse

In [3]:
# driver = webdriver.Chrome(executable_path = "chromedriver-win64/chromedriver.exe")

# url = "https://www.freelancer.com/job/"

# driver.get(url)
# time.sleep(5)
# # Lấy HTML sau khi trang đã tải xong
# html_content = driver.page_source
# # Phân tích HTML bằng BeautifulSoup
# # soup = BeautifulSoup(html_content, 'html.parser')

# driver.quit()
# file_path = "C:/Users/Admin/OneDrive - VNU-HCMUS/Documents/K1N4/UD_PTDLTM/freelancer_jobs.html"

# with open(file_path, "w", encoding="utf-8") as file:
#     file.write(html_content)

In [4]:
# # 1 = web, 2 = Mobile, 6 = Data,  7 = AI


# #  Categories to target
# categories = [2, 3, 6, 7]

# # Create a dictionary to hold lists of tuples (href, job_count) for each category
# category_links = {category_id: [] for category_id in categories}

# # Parse the HTML
# soup = BeautifulSoup(html_content, 'html.parser')

# # Iterate over the categories
# for category_id in categories:
#     category_section = soup.find('section', id=f'category-{category_id}')
#     if category_section:
#         # Find all list items (jobs) in the category
#         job_items = category_section.find_all('li')
        
#         for job_item in job_items:
#             # Find the link inside the job item
#             job_link = job_item.find('a', class_='PageJob-category-link')
#             if job_link:
#                 # Extract the job count using regex
#                 job_text = job_link.get_text(strip=True)
#                 job_count = re.search(r'\((\d+)\)', job_text)
                
#                 if job_count and int(job_count.group(1)) > 0:
#                     job_href = job_link['href']
#                     # Save href and job_count as a tuple (href, job_count)
#                     category_links[category_id].append((job_href, int(job_count.group(1))))

# # Print the lists for each category
# for category_id, href_list in  category_links.items():
#     print(f'Category {category_id} links: {href_list}')


In [5]:
# import json

# # Save category_links to a JSON file
# with open('category_links.json', 'w', encoding='utf-8') as json_file:
#     json.dump(category_links, json_file, ensure_ascii=False, indent=4)


In [6]:
# url_of_each_list = "https://www.freelancer.com/jobs/dot-net/1/"

# headers = {
#         'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
#     }

# # Gửi yêu cầu HTTP GET đến URL với headers
# response = requests.get(url_of_each_list, headers=headers)

# # Kiểm tra xem yêu cầu có thành công hay không
# if response.status_code == 200:
#     html_content = response.text
#     file_path = "C:/Users/Admin/OneDrive - VNU-HCMUS/Documents/K1N4/UD_PTDLTM/joblist.html"
#     with open(file_path, "w", encoding="utf-8") as file:
#         file.write(html_content)
# else:
#     print("failed")

In [7]:
# # - với mỗi category 2,3,6,7 trong json
# #     - lấy ra link và số lượng job
# #     - n_page = số lượng job chia nguyên cho 50
# #     - với mỗi (int)page_i trong n_page
# #     - url = f"https://www.freelancer.com + {link} + {page_i}
# import math

# with open('category_links.json', 'r', encoding='utf-8') as json_file:
#     category_links = json.load(json_file)


# # Dictionary to hold all the URLs for each category
# category_urls = {}

# # Iterate over each category (2, 3, 6, 7)
# for category_id, jobs in category_links.items():
#     urls = []  # List to store URLs for the current category
    
#     for job in jobs:
#         link, job_count = job
        
#         # Calculate the number of pages
#         n_page = math.ceil(job_count / 50)  # Round up to cover all jobs
        
#         # Generate the URLs for each page
#         for page_i in range(1, n_page + 1):
#             url = f"https://www.freelancer.com{link}?page={page_i}"
#             urls.append(url)  # Add the URL to the list
    
#     # Store the URLs list in the dictionary
#     category_urls[category_id] = urls

# # Save the category URLs to a new JSON file
# with open('list_jobs_URL_of_each_category.json', 'w', encoding='utf-8') as json_file:
#     json.dump(category_urls, json_file, ensure_ascii=False, indent=4)


In [8]:
# Get the list of Job URLs from the url page in the list_jobs_URL_of_each_category.json
def get_job_atrributes(url):

    soup=BeautifulSoup(requests.get(url).content, 'html.parser')

    # Get the seo_url through class="JobSearchCard-primary-heading" html
    anchor_tag = soup.find_all('a', class_='JobSearchCard-primary-heading-link')

    # Get the 'href' attribute
    href_list = [tag['href'] for tag in anchor_tag]

    return href_list

#Read file json
def read_json_list_jobs(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return json.load(file)
    
# From list of job in each page, get seo url of each job, and each contest
def get_seo_url_json_list_jobs(list_url):
    contests_seo_urls=[]
    projects_seo_urls=[]
    for url in list_url:
        
        seo_urls=get_job_atrributes(url)
        for seo_url in seo_urls:
            
            #If have 'contest' in url, append to contest list
            if seo_url[0:9]=='/contest/':
                contests_seo_urls.append(seo_url)
            else :
                projects_seo_urls.append(seo_url)
                
    #Remove "/projects/" in url
    projects_seo_urls=[url.replace("/projects/","") for url in projects_seo_urls]
    
    #Apply regex to get the contest id contain 7 digits
    contests_id=[re.search(r'\d{7}',url).group(0) for url in contests_seo_urls]
    
    return projects_seo_urls, contests_id

In [9]:
def create_api_url_request(p_base_url,p_params):
    
    # Encode parameters
    encoded_params = urllib.parse.urlencode(p_params, doseq=True)

    # Construct the full URL
    url = f'{p_base_url}?{encoded_params}'

    # Headers for authorization
    headers = {
        'Authorization': f'Bearer {access_token}'
    }
    
    return url, headers

In [10]:
# Get data from API, input is list of contest id, output is data response from API
def get_contests(p_contests_id):
    
    # Parameters contructed as a dictionary
    params = {
        'contests[]': p_contests_id,
        'job_details':True,
    }
    base_url = 'https://www.freelancer.com/api/contests/0.1/contests/'
    url, headers = create_api_url_request(base_url, params)
    try: 
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f'Error: {e}')
        return None

# Get data from API, input is list of project seo urls, output is data response from API
def get_projects(p_seo_urls):
    # Parameters contructed as a dictionary
    params = {
        'seo_urls[]': p_seo_urls,
        'full_description': True,
        'job_details': True,
    }
    base_url = 'https://www.freelancer.com/api/projects/0.1/projects/'
    url, headers = create_api_url_request(base_url, params)
    try: 
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f'Error: {e}')
        return None
def seo_url_too_long_handler(p_seo_urls):
    # Partition the list of URLs into smaller chunks
    chunk_size = 50
    chunks = [p_seo_urls[i:i + chunk_size] for i in range(0, len(p_seo_urls), chunk_size)]
    # Initialize an empty list to store the responses
    responses = []
    # Iterate over each chunk
    for chunk in chunks:
        # Get the response from the API
        response = get_projects(chunk)
        # Append the response to the list
        responses.append(response)
    return responses

In [11]:
# Get data from API, input is start_id, end_id, category (file list_jobs_URL_of_each_category), output is data response from API
def get_jobs(p_category,p_min,p_max):
    current_file_path=os.getcwd()
    #json.loads(open(, "r", encoding="utf-8").read())
    file_path="/".join((os.path.dirname(os.path.dirname(os.path.dirname(current_file_path))),\
                        "data","utils","list_jobs_URL_of_each_category.json"))
    body_content=read_json_list_jobs(file_path)


    # Get urls from p_min index to p_max index of category p_category
    body_content=body_content[str(p_category)][p_min:p_max]

    projects_seo_urls, contests_id=get_seo_url_json_list_jobs(body_content)

    projects=None
    contests=None
    if len(projects_seo_urls)>0:
        try:
            projects=seo_url_too_long_handler(projects_seo_urls)
        except:
            pass
    if len(contests_id)>0:
        try:
            contests=get_contests(contests_id)
        except:
            pass
    return projects, contests


In [12]:
# Get list of bider of project id from API
def get_bid(project_id):
    # Parameters contructed as a dictionary
    params = {

    }
    base_url = 'https://www.freelancer.com/api/projects/0.1/projects/{}/bids/'.format(project_id)
    url, headers = create_api_url_request(base_url, params)
    try: 
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f'Error: {e}')
        return None
# Get list of entrants of contest id from API (Competition)
def get_entrants(contest_ids:list):
    # Parameters contructed as a dictionary
    params = {
        'contest_ids[]': contest_ids,
    }
    base_url = 'https://www.freelancer.com/api/contests/0.1/entrants/all/'
    url, headers = create_api_url_request(base_url, params)
    try: 
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f'Error: {e}')
        return None

In [13]:
# Convert data from API to dataframe
def to_df_projects_contests(projects,contests,log_status=True):
    df=pd.DataFrame(columns=["job_id","owner_id","type","title","services","skills","description",
                             "date_posted","time_remaining","budget_min","budget_max","budget_currency",
                             "exchange_rate","bid_count","bid_list","average_bid","status","language","url"])
    #Save projects to dataframe
    try: 
        for projects_chunk in projects:
            for project in projects_chunk['result']['projects']:
                if project is None:
                    continue
                
                if log_status:
                    print("INFO: Getting project with id: ",project['id'])
                    
                job_id=project['id']
                owner_id=project['owner_id']
                _type="Project"
                title=project['title']
                services=project['seo_url'].split("/")[0]
                skills=[skill['name'] for skill in project['jobs']]
                description=project['description']
                date_posted=project['submitdate']
                time_free_bids_expire=project['time_free_bids_expire']
                budget_min=project['budget']['minimum']
                budget_max=project['budget']['maximum']
                budget_currency=project['currency']['code']
                exchange_rate=project['currency']['exchange_rate']
                bid_count=project['bid_stats']['bid_count']
                try:
                    bid_list=[bid_user['bidder_id'] for bid_user in get_bid(job_id)['result']['bids']]
                except:
                    bid_list=None
                average_bid=project['bid_stats']['bid_avg']
                status=project['status']
                language=project['language']
                url="https://www.freelancer.com/projects"+project['seo_url']
                #Check if the job_id already exists in the dataframe
                if job_id in df['job_id'].values:
                    continue
                else:
                    new_row_df=pd.DataFrame({"job_id":job_id,"owner_id":owner_id,"type":_type,"title":title,\
                                            "services":services,"skills":[skills],"description":description,\
                                            "date_posted":date_posted,"time_remaining":time_free_bids_expire,\
                                            "budget_min":budget_min,"budget_max":budget_max,"budget_currency":budget_currency,\
                                            "exchange_rate":exchange_rate,"bid_count":bid_count,"bid_list":[bid_list],"average_bid":average_bid,\
                                            "status":status,"language":language,"url":url})
                    df=pd.concat([df,new_row_df],ignore_index=True)
    except:
        pass
    
    
    #Save contests to dataframe
    try: 
        for contest in contests['result']['contests']:
            if contest is None:
                continue
            if log_status:
                print("INFO: Getting contest with id: ",contest['id'])
            job_id=contest['id']
            owner_id=contest['owner_id']
            _type="Contest"
            title=contest['title']
            services=None
            skills=[skill['name'] for skill in contest['jobs']]
            description=contest['description']
            date_posted=contest['time_submitted']
            time_remaining=None
            budget_min=contest['prize']
            budget_max=contest['prize']
            budget_currency=contest['currency']['code']
            exchange_rate=contest['currency']['exchange_rate']
            try:
                bid_list=[bid_user['id'] for bid_user in get_entrants([job_id])['result']['entrants']]
            except:
                bid_list=None
            bid_count=len(bid_list)
            average_bid=None
            status=contest['status']
            language=contest['language']
            url="https://www.freelancer.com/"+contest['seo_url']
            #Check if the job_id already exists in the dataframe
            if job_id in df['job_id'].values:
                continue
            else:
                new_row_df=pd.DataFrame({"job_id":job_id,"owner_id":owner_id,"type":_type,"title":title,\
                                        "services":services,"skills":[skills],"description":description,\
                                        "date_posted":date_posted,"time_remaining":time_remaining,\
                                        "budget_min":budget_min,"budget_max":budget_max,"budget_currency":budget_currency,\
                                        "exchange_rate":exchange_rate,"bid_count":bid_count,"bid_list":[bid_list],"average_bid":average_bid,\
                                        "status":status,"language":language,"url":url})
                df=pd.concat([df,new_row_df],ignore_index=True)
    except:
        pass
    return df


In [14]:
# Save dataframe to csv file (Projects and Contests)
def save_projects_contests_to_csv(df,category,p_min,p_max):
    current_file_path=os.getcwd()
    file_path="/".join((os.path.dirname(os.path.dirname(os.path.dirname(current_file_path))),\
                        "data","raw","freelancer_jobs",f"freelancer_jobs_category_{category}_{p_min}_{p_max}.csv"))
    df.to_csv(file_path,index=False)
    return file_path

In [15]:
# Main function to get data projects and contests from API
def get_data_jobs(category,p_min,p_max,log_status=True):
    projects,contests=get_jobs(category,p_min,p_max)
    df=to_df_projects_contests(projects,contests,log_status)
    file_path=save_projects_contests_to_csv(df,category,p_min,p_max)
    return file_path

In [16]:
# Get list of user (bider and entrants) from csv file
def get_user_id(file_path):
    df=pd.read_csv(file_path)
    return df['bid_list'].values

In [17]:
# Get data from API, input is list of user id, output is data response from API
def get_user(p_user_ids):
     # Parameters contructed as a dictionary
    params = {
        'users[]': p_user_ids,
        'profile_description':True,
        'reputation'  :True,
        'reputation_extra':True,
    }
    base_url = 'https://www.freelancer.com/api/users/0.1/users/'
    url, headers = create_api_url_request(base_url, params)
    try: 
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f'Error: {e}')
        return None

In [18]:
# Convert data from API to dataframe
def to_df_users(data):
    df=pd.DataFrame(columns=["user_id","username","region","overview","ratings","completed_ratings","url"])
    for user in list(data['result']['users'].values()):
        user_id=user['id']
        username=user['username']
        region=user['location']['country']['name']
        overview=user['profile_description']
        ratings=user['reputation']['entire_history']['overall']
        completed_ratings=user['reputation']['entire_history']['completion_rate']
        url="https://www.freelancer.com/u/"+username
        if user_id in df['user_id'].values:
            continue
        else:
            new_row_df=pd.DataFrame({"user_id":user_id,"username":username,"region":region,\
                                    "overview":overview,"ratings":ratings,"completed_ratings":completed_ratings,\
                                    "url":url},index=[0])
        df=pd.concat([df,new_row_df],ignore_index=True)
    return df

In [19]:
# Save dataframe to csv file (Users)
def save_users_to_csv(df,category,p_min,p_max):
    current_file_path=os.getcwd()
    file_path="/".join((os.path.dirname(os.path.dirname(os.path.dirname(current_file_path))),\
                        "data","raw","freelancer_users",f"freelancer_users_category_{category}_{p_min}_{p_max}.csv"))
    df.to_csv(file_path,index=False)
    return file_path

In [20]:
# Main function to get data users from API
def get_data_users(file_jobs_path,p_categories,p_min,p_max,log_status=True):
    list_user_ids=get_user_id(file_jobs_path)
    df=pd.DataFrame(columns=["user_id","username","region","overview","ratings","completed_ratings","url"])
    for i,user_ids in enumerate(list_user_ids):
        try:
            user_ids=eval(user_ids)
            if log_status:
                print("INFO: Getting user info with batch: ",i)
            data=get_user(user_ids)
            new_df=to_df_users(data)
            df=pd.concat([df,new_df],ignore_index=True)
        except:
            print('except')
            pass
        time.sleep(1)
    file_path=save_users_to_csv(df,p_categories,p_min,p_max)
    return file_path

In [ ]:
#Choose category, start_index, end_index in list_jobs_URL_of_each_category.json
category=6
start_index=0
end_index=65

# Get data from API, if don't want to print log_status, set log_status=False
file_jobs_path=get_data_jobs(category,start_index,end_index)
file_users_path=get_data_users(file_jobs_path,category,start_index,end_index)

INFO: Getting project with id:  38746172
INFO: Getting project with id:  38746151
INFO: Getting project with id:  38745195
INFO: Getting project with id:  38745116
INFO: Getting project with id:  38744778
INFO: Getting project with id:  38744587
INFO: Getting project with id:  38743420
INFO: Getting project with id:  38743115
INFO: Getting project with id:  38742821
INFO: Getting project with id:  38742797
INFO: Getting project with id:  38742597
INFO: Getting project with id:  38741397
INFO: Getting project with id:  38741400
INFO: Getting project with id:  38740996
INFO: Getting project with id:  38740883
INFO: Getting project with id:  38740830
INFO: Getting project with id:  38740514
INFO: Getting project with id:  38740061
INFO: Getting project with id:  38739703
INFO: Getting project with id:  38739103
INFO: Getting project with id:  38738988
INFO: Getting project with id:  38737856
INFO: Getting project with id:  38737776
INFO: Getting project with id:  38737253
INFO: Getting pr